In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import validation_curve
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/data-science-london-scikit-learn/train.csv', header=None)
trainLabels = pd.read_csv('/kaggle/input/data-science-london-scikit-learn/trainLabels.csv', header=None)
test = pd.read_csv ('/kaggle/input/data-science-london-scikit-learn/test.csv', header=None)
X = train
y = trainLabels

# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [ ]:
from sklearn.mixture import GaussianMixture
full = np.r_[train, test]
lowest_bic = np.infty
bic = []
n_components_range = range(1,7)
cv_types = ['spherical', 'tied', 'diag', 'full']
best_gmm = None
for cv_type in cv_types:
    for n_components in n_components_range:
        gmm = GaussianMixture(n_components=n_components, covariance_type=cv_type)
        gmm.fit(full)
        bic.append(gmm.aic(full))
        if bic[-1] < lowest_bic:
            lowest_bic = bic[-1]
            best_gmm = gmm
print(best_gmm)

best_gmm.fit(full)
gmm_train = best_gmm.predict_proba(train)
gmm_test = best_gmm.predict_proba(test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

knn = KNeighborsClassifier()

n_neighbors = np.arange(3, 10, 2)
param_grid = dict(n_neighbors=n_neighbors)
grid_search_knn = GridSearchCV(estimator=knn, param_grid=param_grid, cv=10, n_jobs=-1, scoring='accuracy'
                                  ).fit(gmm_train, y.values.ravel())
knn_clf = grid_search_knn.best_estimator_

knn_clf.fit(gmm_train, y.values.ravel())
predict = knn_clf.predict(gmm_test)

predict_df = pd.DataFrame(predict, columns=['Solution'], index=np.arange(1,9001))
predict_df.index.name = 'Id'
predict_df.reset_index(drop=False, inplace=True)

In [ ]:
predict_df.to_csv('submission.csv', index=False)